In [6]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-08-18 17:04:51--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-18 17:04:51 (14.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [11]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-08-18 17:06:58--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.03s   

2024-08-18 17:06:58 (23.7 MB/s) - ‘documents.json’ saved [658332/658332]



In [19]:
import minsearch

In [20]:
import json

In [21]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [22]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [25]:
q = 'the course has already started, can I still enroll?'

In [26]:
index.fit(documents)

In [27]:
boost = {'question': 3.0, 'section': 0.3} ## significa que o campo de question é 3x mais importante e section é menos importante em 0.5x

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

In [21]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [ ]:
boost = {'question': 3.0, 'section': 0.3} ## significa que o campo de question é 3x mais importante e section é menos importante em 0.5x

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, ## me retorna apenas as respostas ref ao curso de data engineering
    boost_dict=boost,
    num_results=5
)

In [1]:
from openai import OpenAI

In [9]:
client = OpenAI()

In [10]:
q = 'the course has already started, can I still enroll?'

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [17]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on several factors:\n\n1. **Institution Policy:** Many institutions have specific enrollment periods, but some may allow late registrations up to a certain date.\n  \n2. **Instructor Approval:** Sometimes, if the course has already started, you may need approval from the instructor to join late.\n\n3. **Availability:** If the course has a cap on the number of students, there may be no spots left.\n\n4. **Catch-Up Work:** Consider whether you'll be able to catch up on the material that has already been covered.\n\nTo proceed, you should:\n1. **Check the Academic Calendar:** Look for any specific deadlines mentioned.\n2. **Contact the Registrar or Enrollment Office:** They can provide information on whether late enrollment is possible and what the process would be.\n3. **Reach Out to the Instructor:** Explain your situation and express your commitment to catching up.\n4. **Review the Course Syllabus:** This will h

In [32]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the question
If the CONTEXT doesn't contain the answer, output NONE


QUESTION: {question}

CONTEXT: {context}
""".strip()


In [29]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [30]:
print(context)

section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you c

In [33]:
prompt = prompt_template.format(question=q, context=context)

In [34]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering the question
If the CONTEXT doesn't contain the answer, output NONE


QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadl

In [36]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [37]:
response.choices[0].message.content

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'